In [2]:
import tensorflow as tf
model = tf.keras.models.load_model('FR4model')

In [7]:
tf.keras.models.save_model(model, "FRmodel.h5")

C:\Users\91702\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [8]:
model = tf.keras.models.load_model('FRmodel.h5')

In [9]:
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np


model = keras.models.load_model('FRmodel.h5')

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    x=tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

# inspect the layers operations inside your frozen graph definition and see the name of its input and output tensors
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

# Save frozen graph from frozen ConcreteFunction to hard drive
# serialize the frozen graph and its text representation to disk.
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="simple_frozen_graph.pb",
                  as_text=False)

#Optional
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="simple_frozen_graph.pbtxt",
                as_text=True)


--------------------------------------------------
Frozen model layers: 
x
model/Conv1/Conv2D/ReadVariableOp/resource
model/Conv1/Conv2D/ReadVariableOp
model/Conv1/Conv2D
model/bn_Conv1/ReadVariableOp/resource
model/bn_Conv1/ReadVariableOp
model/bn_Conv1/ReadVariableOp_1/resource
model/bn_Conv1/ReadVariableOp_1
model/bn_Conv1/FusedBatchNormV3/ReadVariableOp/resource
model/bn_Conv1/FusedBatchNormV3/ReadVariableOp
model/bn_Conv1/FusedBatchNormV3/ReadVariableOp_1/resource
model/bn_Conv1/FusedBatchNormV3/ReadVariableOp_1
model/bn_Conv1/FusedBatchNormV3
model/Conv1_relu/Relu6
model/expanded_conv_depthwise/depthwise/ReadVariableOp/resource
model/expanded_conv_depthwise/depthwise/ReadVariableOp
model/expanded_conv_depthwise/depthwise
model/expanded_conv_depthwise_BN/ReadVariableOp/resource
model/expanded_conv_depthwise_BN/ReadVariableOp
model/expanded_conv_depthwise_BN/ReadVariableOp_1/resource
model/expanded_conv_depthwise_BN/ReadVariableOp_1
model/expanded_conv_depthwise_BN/FusedBatchNormV3

'./frozen_models\\simple_frozen_graph.pbtxt'